In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
#from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [3]:
data = pd.read_csv('./data/retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

Товар __"999999"__ это все товары которые не пользуются популярностью и не входят в 5000 самых  популярных. Нужен для обозначения __"прочие товары"__ и упрощения исходной матрицы (ускорения рвсчетов). Вводя это обозначение мы создаем новый __Мега-товар__ и смещаем рекомендации в его сторону. Удаление этого товара приводит к улучшению работы более осмысленных алгоритмов таких как __knn__, __tfidf__, __cos__.
Значительно ухудшает работу __Popularity-based recommendation__ и __own_purchases__ алгоритмов.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

Сделайте предсказания

In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head(3)

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1


In [7]:
data_train_raw = data_train.copy()

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\rost_\Anaconda3\envs\recomendation_course\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_item_matrix_full = user_item_matrix.copy()


user_item_matrix = user_item_matrix.drop([999999],axis=1)
user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Random Recommendation

In [9]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [10]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(3)

Wall time: 399 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]"


###  Popularity-based recommendation

In [11]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [12]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(3)

Wall time: 109 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]"


### Weighted Random Recommendation

In [13]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
          
    recs = np.random.choice(items_weights['item_id'], n,
              p=items_weights['n_sold'], replace=False)
    
    
    return recs.tolist()

In [14]:
%%time


items_weights = popularity.sort_values('n_sold', ascending=False).head(5000)
items_weights['n_sold'] = np.log(items_weights['n_sold'])
tempo_sum = items_weights['n_sold'].sum()
items_weights['n_sold'] = items_weights['n_sold']/tempo_sum

result['random_recommendation_weighted_count'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(3)

Wall time: 414 ms


,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]"


In [15]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.rename(columns={'sales_value': 'n_sold'}, inplace=True)

popularity.head(3)

,item_id,n_sold
0,202291,81.59
1,397896,2862.41
2,420647,463.81


In [16]:
%%time


items_weights = popularity.sort_values('n_sold', ascending=False).head(5000)
items_weights['n_sold'] = np.log(items_weights['n_sold'])
tempo_sum = items_weights['n_sold'].sum()
items_weights['n_sold'] = items_weights['n_sold']/tempo_sum

result['random_recommendation_weighted_sum'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(3)

Wall time: 393 ms


,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]"


##  Item-Item Recommender / ItemKNN

In [17]:
K_neighbours = 3
K_neighbours = 23

In [18]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [19]:
%%time

model = ItemItemRecommender(K=K_neighbours, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 1.1 s


In [20]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 161 ms


In [21]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]","[1082185, 981760, 995242, 840361, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]","[1082185, 981760, 1098066, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]","[1082185, 981760, 995242, 840361, 1127831]"


### Косинусное сходство и CosineRecommender

In [22]:
%%time

model = CosineRecommender(K=K_neighbours, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 1.24 s


In [23]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 210 ms


In [24]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 840361]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 1098066]"


### TF-IDF взвешивание и TFIDFRecommender

In [25]:
%%time

model = TFIDFRecommender(K=K_neighbours, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 1.13 s


In [26]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

Wall time: 213 ms


In [27]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 840361]","[1082185, 981760, 1127831, 840361, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 1098066, 826249, 840361]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 840361]"


### Трюк

In [28]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

Wall time: 1.07 s


In [29]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

Wall time: 80.8 ms


In [30]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[999999, 6534178, 6533889, 1029743, 6534166]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 840361]","[1082185, 981760, 1127831, 840361, 961554]","[995242, 840361, 904360, 995785, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[999999, 6534178, 6533889, 1029743, 6534166]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 1098066, 826249, 840361]","[826249, 904360, 883404, 866211, 844179]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[999999, 6534178, 6533889, 1029743, 6534166]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 840361]","[981760, 995242, 840361, 904360, 995785]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

Лучше всего из базовых предсказаний сработал "ТРЮК" __own_purchases_rec:         0.17969311132876264__ при не удаленном товаре __999999__. Все остальные подходы (сами по себе) были хуже. Хуже так как слабо учитывают внутреннюю взаимсосвязь данных.

In [31]:
pop_rec             = result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()
random_rec          = result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()
random_weighted_rec_count = result.apply(lambda x: precision_at_k(x['random_recommendation_weighted_count'], x['actual'],  5), axis=1).mean()
random_weighted_rec_sum   = result.apply(lambda x: precision_at_k(x['random_recommendation_weighted_sum'], x['actual'],  5), axis=1).mean()

knn_rec             = result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()
cosine_rec          = result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()
tfidf_rec           = result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()
own_purchases_rec   = result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()


print('random_rec:               ', random_rec)
print('random_weighted_rec_count:', random_weighted_rec_count)
print('random_weighted_rec_sum:  ', random_weighted_rec_sum)
print('cosine_rec:               ', cosine_rec)
print('knn_rec:                  ', knn_rec)
print('tfidf_rec:                ', tfidf_rec)
print('pop_rec:                  ', pop_rec)
print('own_purchases_rec:        ', own_purchases_rec)


random_rec:                0.008129285014691479
random_weighted_rec_count: 0.00852105778648384
random_weighted_rec_sum:   0.006660137120470128
cosine_rec:                0.1525954946131244
knn_rec:                   0.16532810969637612
tfidf_rec:                 0.15670910871694418
pop_rec:                   0.08560235063663076
own_purchases_rec:         0.14273587985634997


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


Бейзлайны - безнадежны... Улучшению не подлежат   
__К = 3__ оказался лучшим.  
Ансамбль трех алгаритмов __(own_purchases, tfidf, knn)__ несколько улучшил прогноз.

In [32]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_raw, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
pop_rec = result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()
print('pop_rec:                  ', pop_rec)

pop_rec:                   0.15523996082272282
Wall time: 387 ms


In [33]:
# перенумеруем пользователей и товары
userids = user_item_matrix_full.index.values
itemids = user_item_matrix_full.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix_full).T.tocsr(), 
          show_progress=True)


result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])
own_purchases_rec   = result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()
print('own_purchases_rec:        ', own_purchases_rec)

  0%|          | 0/5001 [00:00<?, ?it/s]

own_purchases_rec:         0.17969311132876264


In [34]:
pop_rec             = result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()
random_rec          = result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()
random_weighted_rec_count = result.apply(lambda x: precision_at_k(x['random_recommendation_weighted_count'], x['actual'],  5), axis=1).mean()
random_weighted_rec_sum   = result.apply(lambda x: precision_at_k(x['random_recommendation_weighted_sum'], x['actual'],  5), axis=1).mean()

knn_rec             = result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()
cosine_rec          = result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()
tfidf_rec           = result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()
own_purchases_rec   = result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()


#print('random_rec:               ', random_rec)
#print('random_weighted_rec_count:', random_weighted_rec_count)
#print('random_weighted_rec_sum:  ', random_weighted_rec_sum)
print('cosine_rec:               ', cosine_rec)
print('knn_rec:                  ', knn_rec)
print('tfidf_rec:                ', tfidf_rec)
print('pop_rec:                  ', pop_rec)
print('own_purchases_rec:        ', own_purchases_rec)

cosine_rec:                0.1525954946131244
knn_rec:                   0.16532810969637612
tfidf_rec:                 0.15670910871694418
pop_rec:                   0.15523996082272282
own_purchases_rec:         0.17969311132876264


### Ансамбль трех алгаритмов _(own_purchases, tfidf, knn)_ несколько улучшил прогноз.

In [35]:
def collect_list(itemitem_list, tfidf_list, own_purchases_list):
    a=set(own_purchases_list)  
    c=set(itemitem_list)  
    d=set(tfidf_list)
#    collect=(list(c&d)+own_purchases_list)
    collect=list(c&d&a) 
    try:
        collect.remove(999999)
    except:
        pass
    
    last = list(set(own_purchases_list)-set(collect))
    try:
        last.remove(999999)
    except:
        pass        
    
    collect = (collect+last)    
    collect = (collect+tfidf_list)            
    collect = collect[:5]
    
    return collect

In [36]:
result['collect']=result.apply(lambda x: collect_list(x['itemitem'], x['tfidf'], x['own_purchases']), axis=1)

In [37]:
result.apply(lambda x: precision_at_k(x['collect'], x['actual'],  5), axis=1).mean()

0.18285994123408425

In [38]:
result

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_weighted_count,random_recommendation_weighted_sum,itemitem,cosine,tfidf,own_purchases,collect
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966494, 903077, 1084138, 5571444, 13512965]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1091922, 897125, 1067779, 1050883, 934172]","[1111230, 1064755, 9884698, 917384, 906838]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 840361]","[1082185, 981760, 1127831, 840361, 961554]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 995785, 1004906, 1029743, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1095100, 9487927, 8069118, 8090956, 9527161]","[6534178, 6533889, 1029743, 6534166, 1082185]","[926206, 9836526, 850841, 7462562, 936634]","[1136486, 1013999, 1047076, 905913, 855544]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 1098066, 826249, 840361]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 1098066, 6534178, 1127831, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[5586942, 835004, 908408, 947292, 990440]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6602440, 966891, 902167, 937526, 956813]","[1007642, 908846, 825994, 1112761, 1139830]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 840361]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082185, 1127831, 6534178, 1029743, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9523079, 913201, 968057, 974766, 8090956]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1042628, 935993, 1001680, 8090513, 1003616]","[828143, 1043837, 1118623, 5995445, 888404]","[1082185, 981760, 995242, 1098066, 840361]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 840361, 826249]","[999999, 1082185, 1029743, 1127831, 995785]","[1082185, 1127831, 995785, 1029743, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[859426, 5995423, 823356, 829928, 819255]","[6534178, 6533889, 1029743, 6534166, 1082185]","[7410344, 12172355, 1119454, 913201, 1075283]","[1113111, 5569615, 1095068, 856435, 977798]","[1082185, 981760, 1098066, 995242, 840361]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 1098066, 840361, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082185, 1098066, 6534178, 1029743, 1082185]"
...,...,...,...,...,...,...,...,...,...,...,...
2037,2496,[6534178],"[7169034, 1056509, 1011736, 917438, 10149597]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1118598, 1055155, 1137483, 1053690, 1095275]","[1046957, 1079987, 886125, 985716, 202291]","[1082185, 981760, 1098066, 840361, 995242]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 840361, 1098066, 826249]","[999999, 1082185, 1098066, 6534178, 995785]","[1082185, 1098066, 995785, 6534178, 1082185]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[847790, 1107824, 919766, 1096290, 972445]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534480, 1027284, 987480, 831536, 12731685]","[1103903, 988416, 910473, 944317, 876695]","[1082185, 981760, 1098066, 995242, 840361]","[1082185, 981760, 1098066, 826249, 840361]","[1082185, 981760, 1098066, 826249, 840361]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082185, 1098066, 6534178, 1029743, 1082185]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[987941, 1051810, 1083740, 1042702, 1013528]","[6534178, 6533889, 1029743, 6534166, 1082185]","[820291, 14050460, 7025250, 5568729, 8065410]","[932562, 988520, 1080429, 1044759, 1083070]","[1082185, 981760, 1098066, 995242, 840361]","[1082185, 981760, 1098066, 1127831, 826249]","[1082185, 981760, 840361, 1098066, 826249]","[999999, 1082185, 1029743, 1068719, 1106523]","[1082185, 1068719, 1106523, 1029743, 1082185]"
2040,2499,"[867188, 8775

In [39]:
result.shape

(2042, 11)

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

__Реализован несколько другой вариант учета средних предпочтений всех пользователей.__

In [40]:
#data_train_raw = data_train.copy()
data_train = data_train_raw.copy()

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)


top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

#user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(9)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
from scipy.spatial.distance import cdist
import tqdm

def get_N_niarest_vector(matrix, sample_vector, n=5):
    dist_line = cdist(sample_vector, matrix, metric='cosine')
    items = dist_line[0].argsort()[-n:][::-1]
    batch = matrix[items]
    dist_selected = dist_line[0][items]
    return batch, (dist_selected.sum()-1)/(n-1)

In [42]:
def get_user_recomrndations( user_item_matrix, sample_vector, k_neigbours=9, n_recomendation = 5):
    batch, dist = get_N_niarest_vector(user_item_matrix.values, sample_vector, k_neigbours)
    rec_from_yourself = sample_vector
    rec_from_all = user_item_matrix.values.sum(axis=0)/len(user_item_matrix)
    rec_from_neighbours = batch.sum(axis=0)/len(batch)
    
    k1=(dist*5.5)   
    k2=(dist*0.001) 
                   
    rec = rec_from_yourself + rec_from_all/k1 + rec_from_neighbours*k2
    items = rec[0].argsort()[-n_recomendation-1:][::-1]
    
    idx_to_zerro = itemid_to_id[999999]
    items = list(items)
    if idx_to_zerro in items:
        items.remove(idx_to_zerro)
    items =items[:n_recomendation]
    
    return items

In [43]:
k_neigbours=9 
n_recomendation = 5

In [44]:
id_list = result['user_id'].map(userid_to_id).values

rec_list=[]
for i in tqdm.trange(len(id_list)):    
    sample_vector = np.reshape(user_item_matrix.values[id_list[i]],(1,-1))
    item = get_user_recomrndations( user_item_matrix, sample_vector, k_neigbours=k_neigbours, n_recomendation = n_recomendation)
    rec_list.append(item)
#rec_list    

100%|██████████████████████████████████████████████████████████████████████████████| 2042/2042 [06:06<00:00,  5.57it/s]


In [45]:
result_list=[]
for i in tqdm.trange(len(rec_list)):
    item = [id_to_itemid[rec] for rec in rec_list[i]]
    result_list.append(item)
#result_list       

100%|███████████████████████████████████████████████████████████████████████████| 2042/2042 [00:00<00:00, 53874.94it/s]


In [46]:
result['test'] = result_list
#len(result_list), result.shape

In [47]:
result.apply(lambda x: precision_at_k(x['test'], x['actual'],  5), axis=1).mean()

0.4053868756121449

### Пока это лучший  результат: (0.4053868756121449) ###

In [48]:
result['test']

0         [856942, 1082185, 995242, 940947, 9527290]
1        [1092026, 1053690, 951590, 998206, 1082185]
2        [1082185, 1119051, 840361, 1037863, 863447]
3       [1082185, 1122358, 6944571, 1022003, 828867]
4       [1082185, 1133018, 969932, 5569230, 1005186]
                            ...                     
2037      [981760, 916122, 883404, 5569230, 1056509]
2038       [860776, 995785, 1066685, 965719, 896938]
2039    [1082185, 1070820, 1126899, 961554, 1053690]
2040     [1070820, 826249, 5568378, 1082185, 999971]
2041    [1082185, 859237, 6534178, 1065538, 1058997]
Name: test, Length: 2042, dtype: object